## Student 11

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_11_EYE = pd.read_csv('data/STData/11/11_EYE.csv')

In [ ]:
df_11_EYE.head()

In [ ]:
df_11_EYE.shape

In [ ]:
df_11_EYE.columns

In [ ]:
df_11_EYE.info()

In [ ]:
df_11_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_11_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_11_EYE['QuestionKey'].unique()

In [ ]:
df_11_EYE['Timestamp'] = pd.to_datetime(df_11_EYE['Timestamp'])

In [ ]:
df_11_EYE.head(3)

In [ ]:
df_11_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_11_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_11_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_11_EYE.isnull().sum()

In [ ]:
df_11_EYE.dropna(inplace=True)

In [ ]:
df_11_EYE.head()

In [ ]:
df_11_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_11_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_11_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_11_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_11_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_11_EYE['ET_ValidityRight'].unique()

In [ ]:
df_11_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_11_EYE['ET_ValidityLeft'].value_counts().index, y=df_11_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_11_EYE['ET_ValidityRight'].value_counts().index, y=df_11_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_11_EYE['ET_ValidityLeft'] = df_11_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_11_EYE['ET_ValidityRight'] = df_11_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_11_EYE.head(3)

In [ ]:
df_11_EYE.describe()

In [ ]:
df_11_EYE[df_11_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_11_EYE[df_11_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_11_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_11_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilLeft'] == -1].shape[0] / df_11_EYE.shape[0]

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilRight'] == -1].shape[0] / df_11_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_11_EYE[df_11_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_11_EYE[df_11_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_11_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_11_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_11_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_11_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_11_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_11_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_11_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_11_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_11_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_11_EYE['Timestamp'], df_11_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_11_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_11_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_11_EYE[col].fillna(df_11_EYE[col].mean(), inplace=True)

In [ ]:
df_11_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_11_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_11_EYE = pd.read_csv('data/STData/11/11_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_11_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_11_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_11_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_11_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_11_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_11_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_11_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_11_EYE['Timestamp'] = pd.to_datetime(df_11_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_11_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_11_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_11_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_11_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_11_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_11_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_11_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_11_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_11_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_11_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_11_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_11_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_11_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_11_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_11_EYE['ET_ValidityLeft'].value_counts().index, y=df_11_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_11_EYE['ET_ValidityRight'].value_counts().index, y=df_11_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_11_EYE['ET_ValidityLeft'] = df_11_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_11_EYE['ET_ValidityRight'] = df_11_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_11_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_11_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_11_EYE[df_11_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_11_EYE[df_11_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_11_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_11_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilLeft'] == -1].shape[0] / df_11_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_11_EYE[df_11_EYE['ET_PupilRight'] == -1].shape[0] / df_11_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_11_EYE[df_11_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_11_EYE[df_11_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_11_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_11_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_11_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_11_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_11_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_11_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_11_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_11_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_11_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_11_EYE['Timestamp'], df_11_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_11_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_11_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_11_EYE[col].fillna(df_11_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_11_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_11_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_11_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 12

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_12_EYE = pd.read_csv('data/STData/12/12_EYE.csv')

In [ ]:
df_12_EYE.head()

In [ ]:
df_12_EYE.shape

In [ ]:
df_12_EYE.columns

In [ ]:
df_12_EYE.info()

In [ ]:
df_12_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_12_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_12_EYE['QuestionKey'].unique()

In [ ]:
df_12_EYE['Timestamp'] = pd.to_datetime(df_12_EYE['Timestamp'])

In [ ]:
df_12_EYE.head(3)

In [ ]:
df_12_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_12_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_12_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_12_EYE.isnull().sum()

In [ ]:
df_12_EYE.dropna(inplace=True)

In [ ]:
df_12_EYE.head()

In [ ]:
df_12_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_12_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_12_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_12_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_12_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_12_EYE['ET_ValidityRight'].unique()

In [ ]:
df_12_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_12_EYE['ET_ValidityLeft'].value_counts().index, y=df_12_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_12_EYE['ET_ValidityRight'].value_counts().index, y=df_12_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_12_EYE['ET_ValidityLeft'] = df_12_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_12_EYE['ET_ValidityRight'] = df_12_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_12_EYE.head(3)

In [ ]:
df_12_EYE.describe()

In [ ]:
df_12_EYE[df_12_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_12_EYE[df_12_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_12_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_12_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilLeft'] == -1].shape[0] / df_12_EYE.shape[0]

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilRight'] == -1].shape[0] / df_12_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_12_EYE[df_12_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_12_EYE[df_12_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_12_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_12_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_12_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_12_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_12_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_12_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_12_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_12_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_12_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_12_EYE['Timestamp'], df_12_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_12_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_12_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_12_EYE[col].fillna(df_12_EYE[col].mean(), inplace=True)

In [ ]:
df_12_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_12_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_12_EYE = pd.read_csv('data/STData/12/12_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_12_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_12_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_12_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_12_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_12_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_12_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_12_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_12_EYE['Timestamp'] = pd.to_datetime(df_12_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_12_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_12_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_12_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_12_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_12_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_12_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_12_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_12_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_12_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_12_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_12_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_12_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_12_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_12_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_12_EYE['ET_ValidityLeft'].value_counts().index, y=df_12_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_12_EYE['ET_ValidityRight'].value_counts().index, y=df_12_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_12_EYE['ET_ValidityLeft'] = df_12_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_12_EYE['ET_ValidityRight'] = df_12_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_12_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_12_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_12_EYE[df_12_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_12_EYE[df_12_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_12_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_12_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilLeft'] == -1].shape[0] / df_12_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_12_EYE[df_12_EYE['ET_PupilRight'] == -1].shape[0] / df_12_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_12_EYE[df_12_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_12_EYE[df_12_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_12_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_12_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_12_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_12_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_12_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_12_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_12_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_12_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_12_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_12_EYE['Timestamp'], df_12_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_12_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_12_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_12_EYE[col].fillna(df_12_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_12_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_12_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_12_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 13

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_13_EYE = pd.read_csv('data/STData/13/13_EYE.csv')

In [ ]:
df_13_EYE.head()

In [ ]:
df_13_EYE.shape

In [ ]:
df_13_EYE.columns

In [ ]:
df_13_EYE.info()

In [ ]:
df_13_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_13_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_13_EYE['QuestionKey'].unique()

In [ ]:
df_13_EYE['Timestamp'] = pd.to_datetime(df_13_EYE['Timestamp'])

In [ ]:
df_13_EYE.head(3)

In [ ]:
df_13_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_13_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_13_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_13_EYE.isnull().sum()

In [ ]:
df_13_EYE.dropna(inplace=True)

In [ ]:
df_13_EYE.head()

In [ ]:
df_13_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_13_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_13_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_13_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_13_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_13_EYE['ET_ValidityRight'].unique()

In [ ]:
df_13_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_13_EYE['ET_ValidityLeft'].value_counts().index, y=df_13_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_13_EYE['ET_ValidityRight'].value_counts().index, y=df_13_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_13_EYE['ET_ValidityLeft'] = df_13_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_13_EYE['ET_ValidityRight'] = df_13_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_13_EYE.head(3)

In [ ]:
df_13_EYE.describe()

In [ ]:
df_13_EYE[df_13_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_13_EYE[df_13_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_13_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_13_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilLeft'] == -1].shape[0] / df_13_EYE.shape[0]

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilRight'] == -1].shape[0] / df_13_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_13_EYE[df_13_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_13_EYE[df_13_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_13_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_13_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_13_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_13_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_13_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_13_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_13_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_13_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_13_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_13_EYE['Timestamp'], df_13_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_13_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_13_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_13_EYE[col].fillna(df_13_EYE[col].mean(), inplace=True)

In [ ]:
df_13_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_13_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_13_EYE = pd.read_csv('data/STData/13/13_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_13_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_13_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_13_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_13_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_13_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_13_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_13_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_13_EYE['Timestamp'] = pd.to_datetime(df_13_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_13_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_13_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_13_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_13_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_13_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_13_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_13_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_13_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_13_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_13_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_13_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_13_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_13_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_13_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_13_EYE['ET_ValidityLeft'].value_counts().index, y=df_13_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_13_EYE['ET_ValidityRight'].value_counts().index, y=df_13_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_13_EYE['ET_ValidityLeft'] = df_13_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_13_EYE['ET_ValidityRight'] = df_13_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_13_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_13_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_13_EYE[df_13_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_13_EYE[df_13_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_13_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_13_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilLeft'] == -1].shape[0] / df_13_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_13_EYE[df_13_EYE['ET_PupilRight'] == -1].shape[0] / df_13_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_13_EYE[df_13_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_13_EYE[df_13_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_13_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_13_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_13_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_13_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_13_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_13_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_13_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_13_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_13_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_13_EYE['Timestamp'], df_13_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_13_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_13_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_13_EYE[col].fillna(df_13_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_13_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_13_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_13_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 14

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_14_EYE = pd.read_csv('data/STData/14/14_EYE.csv')

In [ ]:
df_14_EYE.head()

In [ ]:
df_14_EYE.shape

In [ ]:
df_14_EYE.columns

In [ ]:
df_14_EYE.info()

In [ ]:
df_14_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_14_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_14_EYE['QuestionKey'].unique()

In [ ]:
df_14_EYE['Timestamp'] = pd.to_datetime(df_14_EYE['Timestamp'])

In [ ]:
df_14_EYE.head(3)

In [ ]:
df_14_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_14_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_14_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_14_EYE.isnull().sum()

In [ ]:
df_14_EYE.dropna(inplace=True)

In [ ]:
df_14_EYE.head()

In [ ]:
df_14_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_14_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_14_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_14_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_14_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_14_EYE['ET_ValidityRight'].unique()

In [ ]:
df_14_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_14_EYE['ET_ValidityLeft'].value_counts().index, y=df_14_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_14_EYE['ET_ValidityRight'].value_counts().index, y=df_14_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_14_EYE['ET_ValidityLeft'] = df_14_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_14_EYE['ET_ValidityRight'] = df_14_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_14_EYE.head(3)

In [ ]:
df_14_EYE.describe()

In [ ]:
df_14_EYE[df_14_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_14_EYE[df_14_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_14_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_14_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilLeft'] == -1].shape[0] / df_14_EYE.shape[0]

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilRight'] == -1].shape[0] / df_14_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_14_EYE[df_14_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_14_EYE[df_14_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_14_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_14_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_14_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_14_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_14_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_14_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_14_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_14_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_14_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_14_EYE['Timestamp'], df_14_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_14_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_14_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_14_EYE[col].fillna(df_14_EYE[col].mean(), inplace=True)

In [ ]:
df_14_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_14_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_14_EYE = pd.read_csv('data/STData/14/14_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_14_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_14_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_14_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_14_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_14_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_14_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_14_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_14_EYE['Timestamp'] = pd.to_datetime(df_14_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_14_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_14_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_14_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_14_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_14_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_14_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_14_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_14_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_14_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_14_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_14_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_14_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_14_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_14_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_14_EYE['ET_ValidityLeft'].value_counts().index, y=df_14_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_14_EYE['ET_ValidityRight'].value_counts().index, y=df_14_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_14_EYE['ET_ValidityLeft'] = df_14_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_14_EYE['ET_ValidityRight'] = df_14_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_14_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_14_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_14_EYE[df_14_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_14_EYE[df_14_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_14_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_14_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilLeft'] == -1].shape[0] / df_14_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_14_EYE[df_14_EYE['ET_PupilRight'] == -1].shape[0] / df_14_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_14_EYE[df_14_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_14_EYE[df_14_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_14_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_14_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_14_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_14_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_14_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_14_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_14_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_14_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_14_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_14_EYE['Timestamp'], df_14_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_14_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_14_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_14_EYE[col].fillna(df_14_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_14_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_14_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_14_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 15

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_15_EYE = pd.read_csv('data/STData/15/15_EYE.csv')

In [ ]:
df_15_EYE.head()

In [ ]:
df_15_EYE.shape

In [ ]:
df_15_EYE.columns

In [ ]:
df_15_EYE.info()

In [ ]:
df_15_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_15_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_15_EYE['QuestionKey'].unique()

In [ ]:
df_15_EYE['Timestamp'] = pd.to_datetime(df_15_EYE['Timestamp'])

In [ ]:
df_15_EYE.head(3)

In [ ]:
df_15_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_15_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_15_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_15_EYE.isnull().sum()

In [ ]:
df_15_EYE.dropna(inplace=True)

In [ ]:
df_15_EYE.head()

In [ ]:
df_15_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_15_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_15_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_15_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_15_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_15_EYE['ET_ValidityRight'].unique()

In [ ]:
df_15_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_15_EYE['ET_ValidityLeft'].value_counts().index, y=df_15_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_15_EYE['ET_ValidityRight'].value_counts().index, y=df_15_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_15_EYE['ET_ValidityLeft'] = df_15_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_15_EYE['ET_ValidityRight'] = df_15_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_15_EYE.head(3)

In [ ]:
df_15_EYE.describe()

In [ ]:
df_15_EYE[df_15_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_15_EYE[df_15_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_15_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_15_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilLeft'] == -1].shape[0] / df_15_EYE.shape[0]

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilRight'] == -1].shape[0] / df_15_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_15_EYE[df_15_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_15_EYE[df_15_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_15_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_15_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_15_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_15_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_15_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_15_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_15_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_15_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_15_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_15_EYE['Timestamp'], df_15_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_15_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_15_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_15_EYE[col].fillna(df_15_EYE[col].mean(), inplace=True)

In [ ]:
df_15_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_15_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_15_EYE = pd.read_csv('data/STData/15/15_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_15_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_15_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_15_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_15_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_15_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_15_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_15_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_15_EYE['Timestamp'] = pd.to_datetime(df_15_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_15_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_15_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_15_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_15_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_15_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_15_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_15_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_15_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_15_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_15_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_15_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_15_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_15_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_15_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_15_EYE['ET_ValidityLeft'].value_counts().index, y=df_15_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_15_EYE['ET_ValidityRight'].value_counts().index, y=df_15_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_15_EYE['ET_ValidityLeft'] = df_15_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_15_EYE['ET_ValidityRight'] = df_15_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_15_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_15_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_15_EYE[df_15_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_15_EYE[df_15_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_15_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_15_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilLeft'] == -1].shape[0] / df_15_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_15_EYE[df_15_EYE['ET_PupilRight'] == -1].shape[0] / df_15_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_15_EYE[df_15_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_15_EYE[df_15_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_15_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_15_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_15_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_15_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_15_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_15_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_15_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_15_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_15_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_15_EYE['Timestamp'], df_15_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_15_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_15_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_15_EYE[col].fillna(df_15_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_15_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_15_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_15_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 16

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_16_EYE = pd.read_csv('data/STData/16/16_EYE.csv')

In [ ]:
df_16_EYE.head()

In [ ]:
df_16_EYE.shape

In [ ]:
df_16_EYE.columns

In [ ]:
df_16_EYE.info()

In [ ]:
df_16_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_16_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_16_EYE['QuestionKey'].unique()

In [ ]:
df_16_EYE['Timestamp'] = pd.to_datetime(df_16_EYE['Timestamp'])

In [ ]:
df_16_EYE.head(3)

In [ ]:
df_16_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_16_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_16_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_16_EYE.isnull().sum()

In [ ]:
df_16_EYE.dropna(inplace=True)

In [ ]:
df_16_EYE.head()

In [ ]:
df_16_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_16_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_16_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_16_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_16_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_16_EYE['ET_ValidityRight'].unique()

In [ ]:
df_16_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_16_EYE['ET_ValidityLeft'].value_counts().index, y=df_16_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_16_EYE['ET_ValidityRight'].value_counts().index, y=df_16_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_16_EYE['ET_ValidityLeft'] = df_16_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_16_EYE['ET_ValidityRight'] = df_16_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_16_EYE.head(3)

In [ ]:
df_16_EYE.describe()

In [ ]:
df_16_EYE[df_16_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_16_EYE[df_16_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_16_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_16_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilLeft'] == -1].shape[0] / df_16_EYE.shape[0]

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilRight'] == -1].shape[0] / df_16_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_16_EYE[df_16_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_16_EYE[df_16_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_16_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_16_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_16_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_16_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_16_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_16_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_16_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_16_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_16_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_16_EYE['Timestamp'], df_16_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_16_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_16_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_16_EYE[col].fillna(df_16_EYE[col].mean(), inplace=True)

In [ ]:
df_16_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_16_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_16_EYE = pd.read_csv('data/STData/16/16_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_16_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_16_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_16_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_16_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_16_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_16_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_16_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_16_EYE['Timestamp'] = pd.to_datetime(df_16_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_16_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_16_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_16_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_16_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_16_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_16_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_16_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_16_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_16_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_16_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_16_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_16_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_16_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_16_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_16_EYE['ET_ValidityLeft'].value_counts().index, y=df_16_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_16_EYE['ET_ValidityRight'].value_counts().index, y=df_16_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_16_EYE['ET_ValidityLeft'] = df_16_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_16_EYE['ET_ValidityRight'] = df_16_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_16_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_16_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_16_EYE[df_16_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_16_EYE[df_16_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_16_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_16_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilLeft'] == -1].shape[0] / df_16_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_16_EYE[df_16_EYE['ET_PupilRight'] == -1].shape[0] / df_16_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_16_EYE[df_16_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_16_EYE[df_16_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_16_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_16_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_16_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_16_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_16_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_16_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_16_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_16_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_16_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_16_EYE['Timestamp'], df_16_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_16_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_16_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_16_EYE[col].fillna(df_16_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_16_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_16_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_16_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 17

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_17_EYE = pd.read_csv('data/STData/17/17_EYE.csv')

In [ ]:
df_17_EYE.head()

In [ ]:
df_17_EYE.shape

In [ ]:
df_17_EYE.columns

In [ ]:
df_17_EYE.info()

In [ ]:
df_17_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_17_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_17_EYE['QuestionKey'].unique()

In [ ]:
df_17_EYE['Timestamp'] = pd.to_datetime(df_17_EYE['Timestamp'])

In [ ]:
df_17_EYE.head(3)

In [ ]:
df_17_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_17_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_17_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_17_EYE.isnull().sum()

In [ ]:
df_17_EYE.dropna(inplace=True)

In [ ]:
df_17_EYE.head()

In [ ]:
df_17_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_17_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_17_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_17_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_17_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_17_EYE['ET_ValidityRight'].unique()

In [ ]:
df_17_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_17_EYE['ET_ValidityLeft'].value_counts().index, y=df_17_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_17_EYE['ET_ValidityRight'].value_counts().index, y=df_17_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_17_EYE['ET_ValidityLeft'] = df_17_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_17_EYE['ET_ValidityRight'] = df_17_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_17_EYE.head(3)

In [ ]:
df_17_EYE.describe()

In [ ]:
df_17_EYE[df_17_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_17_EYE[df_17_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_17_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_17_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilLeft'] == -1].shape[0] / df_17_EYE.shape[0]

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilRight'] == -1].shape[0] / df_17_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_17_EYE[df_17_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_17_EYE[df_17_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_17_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_17_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_17_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_17_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_17_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_17_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_17_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_17_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_17_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_17_EYE['Timestamp'], df_17_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_17_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_17_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_17_EYE[col].fillna(df_17_EYE[col].mean(), inplace=True)

In [ ]:
df_17_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_17_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_17_EYE = pd.read_csv('data/STData/17/17_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_17_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_17_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_17_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_17_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_17_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_17_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_17_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_17_EYE['Timestamp'] = pd.to_datetime(df_17_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_17_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_17_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_17_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_17_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_17_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_17_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_17_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_17_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_17_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_17_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_17_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_17_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_17_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_17_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_17_EYE['ET_ValidityLeft'].value_counts().index, y=df_17_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_17_EYE['ET_ValidityRight'].value_counts().index, y=df_17_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_17_EYE['ET_ValidityLeft'] = df_17_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_17_EYE['ET_ValidityRight'] = df_17_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_17_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_17_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_17_EYE[df_17_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_17_EYE[df_17_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_17_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_17_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilLeft'] == -1].shape[0] / df_17_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_17_EYE[df_17_EYE['ET_PupilRight'] == -1].shape[0] / df_17_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_17_EYE[df_17_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_17_EYE[df_17_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_17_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_17_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_17_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_17_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_17_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_17_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_17_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_17_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_17_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_17_EYE['Timestamp'], df_17_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_17_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_17_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_17_EYE[col].fillna(df_17_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_17_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_17_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_17_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 18

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_18_EYE = pd.read_csv('data/STData/18/18_EYE.csv')

In [ ]:
df_18_EYE.head()

In [ ]:
df_18_EYE.shape

In [ ]:
df_18_EYE.columns

In [ ]:
df_18_EYE.info()

In [ ]:
df_18_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_18_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_18_EYE['QuestionKey'].unique()

In [ ]:
df_18_EYE['Timestamp'] = pd.to_datetime(df_18_EYE['Timestamp'])

In [ ]:
df_18_EYE.head(3)

In [ ]:
df_18_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_18_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_18_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_18_EYE.isnull().sum()

In [ ]:
df_18_EYE.dropna(inplace=True)

In [ ]:
df_18_EYE.head()

In [ ]:
df_18_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_18_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_18_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_18_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_18_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_18_EYE['ET_ValidityRight'].unique()

In [ ]:
df_18_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_18_EYE['ET_ValidityLeft'].value_counts().index, y=df_18_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_18_EYE['ET_ValidityRight'].value_counts().index, y=df_18_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_18_EYE['ET_ValidityLeft'] = df_18_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_18_EYE['ET_ValidityRight'] = df_18_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_18_EYE.head(3)

In [ ]:
df_18_EYE.describe()

In [ ]:
df_18_EYE[df_18_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_18_EYE[df_18_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_18_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_18_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilLeft'] == -1].shape[0] / df_18_EYE.shape[0]

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilRight'] == -1].shape[0] / df_18_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_18_EYE[df_18_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_18_EYE[df_18_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_18_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_18_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_18_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_18_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_18_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_18_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_18_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_18_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_18_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_18_EYE['Timestamp'], df_18_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_18_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_18_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_18_EYE[col].fillna(df_18_EYE[col].mean(), inplace=True)

In [ ]:
df_18_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_18_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_18_EYE = pd.read_csv('data/STData/18/18_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_18_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_18_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_18_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_18_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_18_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_18_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_18_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_18_EYE['Timestamp'] = pd.to_datetime(df_18_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_18_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_18_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_18_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_18_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_18_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_18_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_18_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_18_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_18_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_18_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_18_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_18_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_18_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_18_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_18_EYE['ET_ValidityLeft'].value_counts().index, y=df_18_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_18_EYE['ET_ValidityRight'].value_counts().index, y=df_18_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_18_EYE['ET_ValidityLeft'] = df_18_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_18_EYE['ET_ValidityRight'] = df_18_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_18_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_18_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_18_EYE[df_18_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_18_EYE[df_18_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_18_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_18_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilLeft'] == -1].shape[0] / df_18_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_18_EYE[df_18_EYE['ET_PupilRight'] == -1].shape[0] / df_18_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_18_EYE[df_18_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_18_EYE[df_18_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_18_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_18_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_18_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_18_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_18_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_18_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_18_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_18_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_18_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_18_EYE['Timestamp'], df_18_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_18_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_18_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_18_EYE[col].fillna(df_18_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_18_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_18_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_18_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 19

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_19_EYE = pd.read_csv('data/STData/19/19_EYE.csv')

In [ ]:
df_19_EYE.head()

In [ ]:
df_19_EYE.shape

In [ ]:
df_19_EYE.columns

In [ ]:
df_19_EYE.info()

In [ ]:
df_19_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_19_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_19_EYE['QuestionKey'].unique()

In [ ]:
df_19_EYE['Timestamp'] = pd.to_datetime(df_19_EYE['Timestamp'])

In [ ]:
df_19_EYE.head(3)

In [ ]:
df_19_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_19_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_19_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_19_EYE.isnull().sum()

In [ ]:
df_19_EYE.dropna(inplace=True)

In [ ]:
df_19_EYE.head()

In [ ]:
df_19_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_19_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_19_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_19_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_19_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_19_EYE['ET_ValidityRight'].unique()

In [ ]:
df_19_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_19_EYE['ET_ValidityLeft'].value_counts().index, y=df_19_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_19_EYE['ET_ValidityRight'].value_counts().index, y=df_19_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_19_EYE['ET_ValidityLeft'] = df_19_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_19_EYE['ET_ValidityRight'] = df_19_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_19_EYE.head(3)

In [ ]:
df_19_EYE.describe()

In [ ]:
df_19_EYE[df_19_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_19_EYE[df_19_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_19_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_19_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilLeft'] == -1].shape[0] / df_19_EYE.shape[0]

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilRight'] == -1].shape[0] / df_19_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_19_EYE[df_19_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_19_EYE[df_19_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_19_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_19_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_19_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_19_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_19_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_19_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_19_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_19_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_19_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_19_EYE['Timestamp'], df_19_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_19_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_19_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_19_EYE[col].fillna(df_19_EYE[col].mean(), inplace=True)

In [ ]:
df_19_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_19_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_19_EYE = pd.read_csv('data/STData/19/19_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_19_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_19_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_19_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_19_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_19_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_19_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_19_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_19_EYE['Timestamp'] = pd.to_datetime(df_19_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_19_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_19_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_19_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_19_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_19_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_19_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_19_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_19_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_19_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_19_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_19_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_19_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_19_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_19_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_19_EYE['ET_ValidityLeft'].value_counts().index, y=df_19_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_19_EYE['ET_ValidityRight'].value_counts().index, y=df_19_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_19_EYE['ET_ValidityLeft'] = df_19_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_19_EYE['ET_ValidityRight'] = df_19_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_19_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_19_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_19_EYE[df_19_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_19_EYE[df_19_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_19_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_19_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilLeft'] == -1].shape[0] / df_19_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_19_EYE[df_19_EYE['ET_PupilRight'] == -1].shape[0] / df_19_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_19_EYE[df_19_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_19_EYE[df_19_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_19_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_19_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_19_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_19_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_19_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_19_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_19_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_19_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_19_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_19_EYE['Timestamp'], df_19_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_19_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_19_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_19_EYE[col].fillna(df_19_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_19_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_19_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_19_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Student 20

In [ ]:
%load_ext cudf

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_20_EYE = pd.read_csv('data/STData/20/20_EYE.csv')

In [ ]:
df_20_EYE.head()

In [ ]:
df_20_EYE.shape

In [ ]:
df_20_EYE.columns

In [ ]:
df_20_EYE.info()

In [ ]:
df_20_EYE.isnull().sum()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_20_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.  
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.  
- These missing values in `QuestionKey` require additional investigation and context-aware handling.  

In [ ]:
df_20_EYE['QuestionKey'].unique()

In [ ]:
df_20_EYE['Timestamp'] = pd.to_datetime(df_20_EYE['Timestamp'])

In [ ]:
df_20_EYE.head(3)

In [ ]:
df_20_EYE['QuestionKey'].fillna('None', inplace=True)

In [ ]:
df_20_EYE['QuestionKey'].value_counts()

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_20_EYE.isnull(), cmap='viridis')
plt.show()

In [ ]:
df_20_EYE.isnull().sum()

In [ ]:
df_20_EYE.dropna(inplace=True)

In [ ]:
df_20_EYE.head()

In [ ]:
df_20_EYE['Row'].unique()

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_20_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

In [ ]:
df_20_EYE.drop('Row', axis=1, inplace=True)

In [ ]:
df_20_EYE['ET_ValidityLeft'].unique()

In [ ]:
df_20_EYE['ET_ValidityLeft'].value_counts()

In [ ]:
df_20_EYE['ET_ValidityRight'].unique()

In [ ]:
df_20_EYE['ET_ValidityRight'].value_counts()

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_20_EYE['ET_ValidityLeft'].value_counts().index, y=df_20_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_20_EYE['ET_ValidityRight'].value_counts().index, y=df_20_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

In [ ]:
df_20_EYE['ET_ValidityLeft'] = df_20_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_20_EYE['ET_ValidityRight'] = df_20_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

In [ ]:
df_20_EYE.head(3)

In [ ]:
df_20_EYE.describe()

In [ ]:
df_20_EYE[df_20_EYE['ET_ValidityLeft'] == 1].shape

In [ ]:
df_20_EYE[df_20_EYE['ET_ValidityRight'] == 1].shape

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_20_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_20_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilLeft'] == -1].shape

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilRight'] == -1].shape

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilLeft'] == -1].shape[0] / df_20_EYE.shape[0]

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilRight'] == -1].shape[0] / df_20_EYE.shape[0]

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_20_EYE[df_20_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_20_EYE[df_20_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

In [ ]:
df_20_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

In [ ]:
df_20_EYE.head()

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_20_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_20_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

In [ ]:
valid_left_ratio  = 1 - df_20_EYE['ET_ValidityLeft'].mean()
valid_right_ratio = 1 - df_20_EYE['ET_ValidityRight'].mean()

In [ ]:
valid_left_ratio

In [ ]:
valid_right_ratio

In [ ]:
df_20_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_20_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
df_20_EYE.columns

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_20_EYE['Timestamp'], df_20_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_20_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
df_20_EYE.replace({-1: np.nan}, inplace=True)

In [ ]:
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_20_EYE[col].fillna(df_20_EYE[col].mean(), inplace=True)

In [ ]:
df_20_EYE.head()

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_20_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Eye Tracking Data Analysis

This notebook performs exploratory data analysis and cleaning on eye-tracking data.

## Data Loading and Initial Inspection

In [ ]:
%load_ext cudf

Import necessary libraries for data manipulation, analysis, and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import libraries for datashader, which can be used for visualizing large datasets.

In [ ]:
import datashader as ds
import datashader.transfer_functions as tf

Set pandas display options to show all columns.

In [ ]:
pd.set_option('display.max_columns', None)

Load the eye-tracking data from the specified CSV file into a pandas DataFrame.

In [ ]:
df_20_EYE = pd.read_csv('data/STData/20/20_EYE.csv')

Display the first few rows of the DataFrame to get an initial look at the data structure and content.

In [ ]:
df_20_EYE.head()

Check the dimensions (number of rows and columns) of the DataFrame.

In [ ]:
df_20_EYE.shape

List the names of all columns in the DataFrame.

In [ ]:
df_20_EYE.columns

Display concise information about the DataFrame, including the data types of each column and the number of non-null values. This helps identify columns with missing data.

In [ ]:
df_20_EYE.info()

Calculate and display the number of missing values in each column.

In [ ]:
df_20_EYE.isnull().sum()

Visualize the distribution of missing values using a heatmap. This provides a visual representation of which columns have missing data and the extent of missingness.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_20_EYE.isnull(), cmap='viridis')
plt.show()

# Notes & Observations

- We observe many **null** (or missing) values in the `QuestionKey` columns.
- The nulls in the `QuestionKey` column may not represent “true” nulls. Rather, they follow interval patterns, suggesting that during those periods no question was displayed.
- These missing values in `QuestionKey` require additional investigation and context-aware handling.

Display the unique values in the `QuestionKey` column to understand the different types of questions or states recorded.

In [ ]:
df_20_EYE['QuestionKey'].unique()

Convert the `Timestamp` column to datetime objects. This is crucial for any time series analysis or manipulation.

In [ ]:
df_20_EYE['Timestamp'] = pd.to_datetime(df_20_EYE['Timestamp'])

Display the first few rows again to see the effect of the timestamp conversion.

In [ ]:
df_20_EYE.head(3)

Fill the missing values in the `QuestionKey` column with the string 'None'. This helps to explicitly mark periods where no question was active.

In [ ]:
df_20_EYE['QuestionKey'].fillna('None', inplace=True)

Count the occurrences of each unique value in the `QuestionKey` column after filling missing values.

In [ ]:
df_20_EYE['QuestionKey'].value_counts()

Display the heatmap of missing values again to confirm that the missing values in `QuestionKey` have been handled.

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df_20_EYE.isnull(), cmap='viridis')
plt.show()

Recalculate and display the number of missing values per column to confirm the changes after handling `QuestionKey`.

In [ ]:
df_20_EYE.isnull().sum()

Drop rows that still contain any missing values. This is done after handling `QuestionKey` separately.

In [ ]:
df_20_EYE.dropna(inplace=True)

Display the first few rows after dropping rows with missing values.

In [ ]:
df_20_EYE.head()

Display the unique values in the `Row` column.

In [ ]:
df_20_EYE['Row'].unique()

Plot a histogram of the `Row` column to visualize its distribution.

In [ ]:
plt.figure(figsize=(8,6))
sns.histplot(df_20_EYE['Row'])
plt.show()

# Notes & Observations

- The `Row` column appears to be a simple row index and does not provide meaningful information relevant to the eye-tracking data itself. Therefore, it can be dropped.

Drop the `Row` column as it is not relevant for the analysis.

In [ ]:
df_20_EYE.drop('Row', axis=1, inplace=True)

Display the unique values in the `ET_ValidityLeft` column.

In [ ]:
df_20_EYE['ET_ValidityLeft'].unique()

Count the occurrences of each unique value in the `ET_ValidityLeft` column.

In [ ]:
df_20_EYE['ET_ValidityLeft'].value_counts()

Display the unique values in the `ET_ValidityRight` column.

In [ ]:
df_20_EYE['ET_ValidityRight'].unique()

Count the occurrences of each unique value in the `ET_ValidityRight` column.

In [ ]:
df_20_EYE['ET_ValidityRight'].value_counts()

Visualize the counts of valid and invalid data for both left and right eyes using bar plots.

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.barplot(x=df_20_EYE['ET_ValidityLeft'].value_counts().index, y=df_20_EYE['ET_ValidityLeft'].value_counts().values)
plt.title('Count of ET_ValidityLeft')
plt.xlabel('Validity')
plt.ylabel('Count')


plt.subplot(1, 2, 2)
sns.barplot(x=df_20_EYE['ET_ValidityRight'].value_counts().index, y=df_20_EYE['ET_ValidityRight'].value_counts().values)
plt.title('Count of ET_ValidityRight')
plt.xlabel('Validity')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Notes & Observations

- The `ET_ValidityLeft` and `ET_ValidityRight` columns indicate the validity of the eye-tracking data for the left and right eye, respectively.
- Based on the value counts and the bar plots, it appears that a value of `0.0` represents valid eye-tracking data, while a value of `4.0` represents invalid data.
- Although the amount of invalid data is relatively small, removing these rows could introduce unwanted patterns or gaps in the time series data.
- Therefore, we will keep the data and replace the value `4.0` with `1.0` in both `ET_ValidityLeft` and `ET_ValidityRight` columns. This will indicate to a machine learning model that the eye tracker had invalid data at those specific points in time while maintaining the integrity of the time series.

Define a mapping to convert validity values from `0.0` and `4.0` to `0` and `1`.

In [ ]:
validity_map = {4.0: 1.0, 0.0: 0.0}

Apply the mapping to the `ET_ValidityLeft` and `ET_ValidityRight` columns and convert the data type to integer.

In [ ]:
df_20_EYE['ET_ValidityLeft'] = df_20_EYE['ET_ValidityLeft'].map(validity_map).astype(np.int8)
df_20_EYE['ET_ValidityRight'] = df_20_EYE['ET_ValidityRight'].map(validity_map).astype(np.int8)

Display the first few rows to see the updated validity columns.

In [ ]:
df_20_EYE.head(3)

Display descriptive statistics for the DataFrame, including count, mean, standard deviation, min, max, and quartiles for numeric columns.

In [ ]:
df_20_EYE.describe()

Check the number of rows where `ET_ValidityLeft` is 1 (invalid data for the left eye).

In [ ]:
df_20_EYE[df_20_EYE['ET_ValidityLeft'] == 1].shape

Check the number of rows where `ET_ValidityRight` is 1 (invalid data for the right eye).

In [ ]:
df_20_EYE[df_20_EYE['ET_ValidityRight'] == 1].shape

Visualize the distribution of -1 and 1 values across columns using heatmaps. This helps identify columns with placeholder or indicator values.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_20_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_20_EYE == 1, cmap='viridis')
plt.title('Heatmap of 1 Values')

plt.tight_layout()
plt.show()

Check the number of rows where `ET_PupilLeft` is -1.

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilLeft'] == -1].shape

Check the number of rows where `ET_PupilRight` is -1.

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilRight'] == -1].shape

Calculate the ratio of rows with -1 values in the `ET_PupilLeft` column to the total number of rows.

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilLeft'] == -1].shape[0] / df_20_EYE.shape[0]

Calculate the ratio of rows with -1 values in the `ET_PupilRight` column to the total number of rows.

In [ ]:
df_20_EYE[df_20_EYE['ET_PupilRight'] == -1].shape[0] / df_20_EYE.shape[0]

Visualize the distribution of -1 values specifically for rows where validity is 1 (invalid data).

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_20_EYE[df_20_EYE['ET_ValidityLeft'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_20_EYE[df_20_EYE['ET_ValidityRight'] == 1] == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

# Notes & Observations

- The heatmaps reveal the distribution of -1 values across different columns.
- It is evident that the `-1` values are not randomly scattered but appear in specific columns, notably `ET_GazeLeftx`, `ET_GazeLefty`, `ET_GazeRightx`, `ET_GazeRighty`, `ET_PupilLeft`, `ET_PupilRight`, `ET_DistanceLeft`, `ET_DistanceRight`, `ET_CameraLeftX`, `ET_CameraLeftY`, `ET_CameraRightX`, and `ET_CameraRightY`.
- These `-1` values often coincide with instances where `ET_ValidityLeft` or `ET_ValidityRight` is 1, indicating invalid eye-tracking data. This suggests that `-1` is used as a placeholder for missing or invalid measurements in these columns when the eye tracker is not providing valid data for a particular eye.
- Given that over 70% of the data in the `ET_PupilLeft` and `ET_PupilRight` columns is marked as invalid (-1), these columns may not be suitable for analysis and could be considered for dropping.

Drop the `ET_PupilLeft` and `ET_PupilRight` columns due to the high percentage of invalid data.

In [ ]:
df_20_EYE.drop(['ET_PupilLeft', 'ET_PupilRight'], axis=1, inplace=True)

Display the first few rows after dropping the pupil columns.

In [ ]:
df_20_EYE.head()

Visualize the distribution of -1 and 1 values again after dropping the pupil columns.

In [ ]:
plt.figure(figsize=(18, 8))

plt.subplot(1, 2, 1)
sns.heatmap(df_20_EYE == -1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.subplot(1, 2, 2)
sns.heatmap(df_20_EYE == 1, cmap='viridis')
plt.title('Heatmap of -1 Values')

plt.tight_layout()
plt.show()

Calculate the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio  = 1 - df_20_EYE['ET_ValidityLeft'].mean()

Display the ratio of valid data points for the left eye.

In [ ]:
valid_left_ratio

Calculate the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio = 1 - df_20_EYE['ET_ValidityRight'].mean()

Display the ratio of valid data points for the right eye.

In [ ]:
valid_right_ratio

Display the first few rows of the DataFrame.

In [ ]:
df_20_EYE.head()

Visualize the distributions of numeric columns using histograms with Kernel Density Estimate (KDE) plots. This helps understand the shape and spread of the data in each numeric column.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_20_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

List the names of the columns in the DataFrame.

In [ ]:
df_20_EYE.columns

Define a list of columns to be plotted over time.

In [ ]:
cols = ['ET_GazeLeftx', 'ET_GazeLefty',
       'ET_GazeRightx', 'ET_GazeRighty', 'ET_TimeSignal', 'ET_DistanceLeft',
       'ET_DistanceRight', 'ET_CameraLeftX', 'ET_CameraLeftY',
       'ET_CameraRightX', 'ET_CameraRightY', 'ET_ValidityLeft',
       'ET_ValidityRight']

Plot each of the selected columns against the timestamp to visualize their trends and patterns over time. Markdown cells are added before each plot for better readability.

In [ ]:
from IPython.display import display, Markdown

for col in cols:
    # Add a markdown cell before each plot for better separation and labeling
    display(Markdown(f'### {col} over Time'))
    plt.figure(figsize=(16, 10))
    plt.plot(df_20_EYE['Timestamp'], df_20_EYE[col])
    plt.xlabel("Timestamp") # Add x-axis label
    plt.ylabel(col) # Add y-axis label
    plt.show()

Visualize the distributions of numeric columns using boxplots. This helps identify potential outliers and the spread of the data.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(numeric_cols) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(numeric_cols):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.boxplot(df_20_EYE[col])
    plt.title(f'Boxplot of {col}')
    plt.xlabel(col)

plt.tight_layout()
plt.show()

Replace the placeholder value -1 with NaN (Not a Number) in the DataFrame. This is done before imputation.

In [ ]:
df_20_EYE.replace({-1: np.nan}, inplace=True)

Impute the missing values (NaN) in the numeric columns with the mean of each column.

In [ ]:
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns

for col in numeric_cols:
    df_20_EYE[col].fillna(df_20_EYE[col].mean(), inplace=True)

Display the first few rows after imputing missing values.

In [ ]:
df_20_EYE.head()

Visualize the distributions of numeric columns again using histograms with KDE plots after imputation. This helps assess the impact of imputation on the data distributions.

In [ ]:
# Select only the numeric columns for plotting histograms, excluding time-related columns
numeric_cols = df_20_EYE.select_dtypes(include=np.number).columns
cols_to_plot = [col for col in numeric_cols if col not in ['UnixTime']]

# Calculate the number of rows and columns for the grid
n_cols = 4  # You can adjust the number of columns as needed
n_rows = (len(cols_to_plot) + n_cols - 1) // n_cols

plt.figure(figsize=(n_cols * 5, n_rows * 4)) # Adjust figure size as needed

for i, col in enumerate(cols_to_plot):
    plt.subplot(n_rows, n_cols, i + 1)
    sns.histplot(df_20_EYE[col], kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()